### Teoría

#### Define brevemente qué es el aprendizaje por refuerzo. ¿Qué diferencias hay entre aprendizaje supervisado, no supervisado y por refuerzo?



Definiendo brevemente, el aprendizaje por refuerzo es un método con el que se pretende optimizar el comportamiento de un agente autónomo para maximizar la recompensa que le asignemos, es decir, se *premia* al modelo cuanto más se acerque a la solución deseada durante una serie de iteraciones.

- El aprendizaje supervisado consiste en realizar inferencias, teniendo acceso a datos con los que previamente entrenar nuestro modelo. 
- En no supervisado no tenemos esa suerte, y se necesita atacar el problema de otra manera (clustering con similitudes y distancias). Las técnicas de no supervisado a menudo se convierten en el preludio de técnicas de supervisado (porque no podemos realizar Supervised Learning si no tenemos datos con los que trabajar). 
- Finalmente, podemos decir que en RL tampoco tenemos datos para entrenar y el problema es diferente: Tenemos un objetivo y queremos encontrar la manera más óptima de llegar a él.

#### Define con tus palabras los conceptos de Entorno, Agente, Recompensa, Estado y Observación.

- Entorno: El universo de nuestro problema. Tiene sus propias características y restricciones. 
- Agente: El modelo con el que estamos trabajando.
- Recompensa: Es cómo premiamos al agente cuando se acerca a la solución que consideramos satisfactoria. 
- Observación: Los parámetros el proyecto en un momento dado del espacio-tiempo del sistema.
- Estado: Parámetros que involucran a nuestro agente y el entorno en un momento dado.

#### Dependiendo del algoritmo de aprendizaje por refuerzo que se use, ¿qué clasificaciones podemos encontrar? Coméntalas brevemente.

Según hemos visto, una buena primera clasificación pueden ser los algoritmos basados en la estrategia y los algoritmos basados en el modelo.

- Estrategia: Determinan qué acción tomar en un estado. Se pueden clasificar en On-Policy (se intenta evaluar y mejorar la misma policy que el agente está usando para elegir una acción) y Off-Policy (la policy que se intenta mejorar es diferente de la usada para seleccionar la acción, como con Q learning).

- Modelo: Existen dinámicas en el entorno que conocemos y las acciones que toma el agente están condicionadas por las mismas. 

#### Lista tres diferencias entre los algoritmos de DQN y Policy Gradient

- Los algoritmos Policy Gradient buscan la mejor policy optimizando la recompensa.
- Los Policy Gradient suelen tener una varianza alta.
- Para entornos contínuos.
- DQNs alcanzan la policy óptima aprendiendo cuáles son las mejores value functions. Alcanzan el mismo objetivo, de forma indirecta.
- Suelen ser más rápidos y estables.

### Práctica

Algunas consideraciones a tener en cuenta:

- El entorno sobre el que trabajaremos será _SpaceInvaders-v0_ y el algoritmo que usaremos será _DQN_.

- Para nuestro ejercicio, una solución óptima será alcanzar una media de recompensa por encima de 16 puntos. Para medir si hemos conseguido llegar a la solución óptima, la media de la recompensa se calculará a partir del código de test en la última celda del notebook.

Este bloque práctico consta de tres partes:

   1) Implementar la red neuronal que se usará en la solución
    
   2) Seleccionar los hiperparámetros adecuados para las distintas piezas de la solución DQN
    
   3) Justificar la respuesta en relación a los resultados obtenidos

IMPORTANTE:

- Si no se consigue una puntuación óptima, responder sobre la mejor puntuación obtenida.

- Para entrenamientos largos, recordad que podéis usar checkpoints de vuestros modelos para retomar los entrenamientos. En este caso, recordad cambiar los parámetros adecuadamente (sobre todo los relacionados con el proceso de exploración).

- Si usáis Google Colab, recordad usar las versiones de Tensorflow==1.13.1, Keras==2.2.4 y keras-rl==0.4.2

In [ ]:
'''Just saying, Tensorflow & keras management of versions are something terrible'''

# !pip uninstall tensorflow
# !pip uninstall keras
!pip install tensorflow==1.13.1
!pip install keras==2.2.4

# Uncomment this line for installing keras-rl on Google collaboratory
!pip install keras-rl

     |████████████████████████████████| 92.5MB 94kB/s 
     |████████████████████████████████| 368kB 41.7MB/s 
     |████████████████████████████████| 3.2MB 32.7MB/s 
  Found existing installation: tensorflow-estimator 1.15.0
    Uninstalling tensorflow-estimator-1.15.0:
      Successfully uninstalled tensorflow-estimator-1.15.0
  Found existing installation: tensorboard 2.1.0
    Uninstalling tensorboard-2.1.0:
      Successfully uninstalled tensorboard-2.1.0
  Found existing installation: tensorflow 2.1.0
    Uninstalling tensorflow-2.1.0:
      Successfully uninstalled tensorflow-2.1.0
     |████████████████████████████████| 317kB 5.3MB/s 
  Found existing installation: Keras 2.3.1
    Uninstalling Keras-2.3.1:
      Successfully uninstalled Keras-2.3.1


In [ ]:
from __future__ import division

from PIL import Image
import numpy as np
import gym

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution2D, Permute
from keras.optimizers import Adam
import keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

Using TensorFlow backend.


In [ ]:
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4

env_name = 'SpaceInvaders-v0'
env = gym.make(env_name)

np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

In [ ]:
class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')

    def process_state_batch(self, batch):
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

1) Implementación de la red neuronal

In [ ]:
input_shape = (WINDOW_LENGTH,)+INPUT_SHAPE

# this structure comes after some research on the internet, mainly https://yilundu.github.io/2016/12/24/Deep-Q-Learning-on-Space-Invaders.html
model = Sequential()
model.add(Permute((2,3,1), input_shape=input_shape))
model.add(Convolution2D(32, 8, 8, subsample=(4, 4)))
model.add(Activation('relu'))
model.add(Convolution2D(64, 4, 4, subsample=(2, 2)))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3, subsample=(1,1)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('sigmoid'))

print (input_shape)
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
(4, 84, 84)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute_1 (Permute)          (None, 84, 84, 4)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 32)        8224      
_________________________________________________________________
activation_1 (Activation)    (None, 20, 20, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 64)          32832     
_________________________________________________________________
activation_2 (Activation)    (None, 9, 9, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 64)          36928     
__________________________________________________________

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (8, 8), strides=(4, 4))`
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (4, 4), strides=(2, 2))`
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), strides=(1, 1))`
  # Remove the CWD from sys.path while we load stuff.


2) Selección de hiperparámetros para la solución DQN

In [ ]:
# TODO - Select the parameters for the memory

memory = SequentialMemory(limit=200000, 
                          window_length=WINDOW_LENGTH) 
                          #https://www.oreilly.com/library/view/keras-deep-learning/9781788621755/8faadfd1-d5ae-44fc-bb71-9861c77262a5.xhtml
processor = AtariProcessor()

In [ ]:
# TODO - Select the parameters for the policy
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), 
                              attr='eps',
                              value_max=1., 
                              value_min=.1, 
                              value_test=.05,
                              nb_steps=200000)

In [ ]:
# TODO - Select the parameters for the Agent and the Optimizer

dqn = DQNAgent(model=model, 
               nb_actions=nb_actions, 
               policy=policy,
               memory=memory, 
               processor=processor,
               nb_steps_warmup=50000, 
               gamma=0.99,
               target_model_update=10000,
               train_interval=20)

dqn.compile(Adam(lr=.0005), metrics=['mae'])

In [9]:
# Training part
weights_filename = 'dqn_{}_weights.h5f'.format(env_name)
checkpoint_weights_filename = 'dqn_' + env_name + '_weights_{step}.h5f'
log_filename = 'dqn_{}_log.json'.format(env_name)
callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=250000)]
callbacks += [FileLogger(log_filename, interval=100)]

# TODO - Select the parameters for the method "fit"
dqn.fit(env, callbacks=callbacks, 
        nb_steps=900000, 
        log_interval=10000, 
        visualize=False)

dqn.save_weights(weights_filename, overwrite=True)

Training for 900000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 55s 5ms/step - reward: 0.0136
12 episodes - episode_reward: 10.583 [5.000, 21.000] - ale.lives: 2.092

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 57s 6ms/step - reward: 0.0133
14 episodes - episode_reward: 9.929 [2.000, 24.000] - ale.lives: 2.079

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 57s 6ms/step - reward: 0.0147
13 episodes - episode_reward: 10.692 [5.000, 22.000] - ale.lives: 2.061

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 55s 6ms/step - reward: 0.0136
14 episodes - episode_reward: 10.000 [6.000, 15.000] - ale.lives: 2.055

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 55s 6ms/step - reward: 0.0164
14 episodes - episode_reward: 11.857 [5.000, 22.000] - ale.lives: 2.022

Interval 6 (50000 steps performed)
   19/10000 [..

In [10]:
# Testing part to calculate the mean reward
weights_filename = 'dqn_{}_weights.h5f'.format(env_name)
dqn.load_weights(weights_filename)
dqn.test(env, nb_episodes=10, visualize=False)

Testing for 10 episodes ...
Episode 1: reward: 15.000, steps: 620
Episode 2: reward: 16.000, steps: 951
Episode 3: reward: 16.000, steps: 707
Episode 4: reward: 17.000, steps: 1023
Episode 5: reward: 18.000, steps: 904
Episode 6: reward: 12.000, steps: 411
Episode 7: reward: 23.000, steps: 1019
Episode 8: reward: 9.000, steps: 509
Episode 9: reward: 26.000, steps: 998
Episode 10: reward: 14.000, steps: 533


#### 3) Justificación de los parámetros seleccionados y de los resultados obtenidos

- Una vez construido el modelo, he intentado seguir las indicaciones de clase para modificar los hiperparámetros. Si bien es cierto que después he probado un par de configuraciones con parámetros ligeramente diferentes de forma estocástica.
- Se ha conseguido superar el objetivo de reward.